# Usage: scenario analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lisphilar/covid19-sir/blob/master/example/usage_quick.ipynb)

This is a quick tour of CovsirPhy. Details scenario analysis will be explained.
"Scenario analysis" means that we calculate the number of cases in the future phases with some sets of ODE parameter values. With this analysis, we can estimate the impact of our activities against the outbreak on the number of cases.

### Preparation
Prepare the packages.

In [ ]:
# !pip install covsirphy --upgrade
from pprint import pprint
import covsirphy as cs
cs.__version__

### Dataset preparation
Download the datasets to "../input" directory and load them.  
Please refer to [Usage: datasets](https://lisphilar.github.io/covid19-sir/usage_dataset.html) for the details.

In [ ]:
loader = cs.DataLoader("../input")
# The number of cases and population values
jhu_data = loader.jhu()
# Government Response Tracker (OxCGRT)
oxcgrt_data = loader.oxcgrt()
# The number of tests
pcr_data = loader.pcr()
# The number of vaccinations
vaccine_data = loader.vaccine()
# Mobility data
mobility_data = loader.mobility()

From development version 2.22.0, we can collect datasets and get them as dictionary with `DataLoader.collect()`.

```Python
data_dict = loader.collect()
snl = cs.Scenario(country="Japan", province=None)
snl.register(**data_dict)
```

### Start scenario analysis

As an example, we will analysis the number of cases in Japan. `covsirphy.Scenario` is the interface for analysis. Please specify the area (country: required, province: optional) when creating the instance and register the datasets with `Scenario.register()`. As the extra datasets, we can select `OxCGRTData`, `PCRData` and `VaccineData`.

In [ ]:
# Specify country and province (optional) names
snl = cs.Scenario(country="Japan", province=None)
# Register datasets
snl.register(jhu_data, extras=[oxcgrt_data, pcr_data, vaccine_data])

We call `JHUData` as "the main datasets" because they are required to calculate the number of susceptible/infected/recovered/fatal cases. These variables are used in SIR-F model.  
The other datasets are called as "the extra datasets" and they will be used to predict the future parameter values of SIR-F model for forecasting the number of cases with some scenarios.

Additional information:  

- Details of the datasets: [Usage: datasets](https://lisphilar.github.io/covid19-sir/usage_dataset.html)
- Details of SIR-F model: [Usage: SIR-derived models](https://lisphilar.github.io/covid19-sir/usage_theoretical.html)

#### Display/save figures
We have interactive mode and script mode to display/save figures.

When use use interactive shells, including Jupyter Notebook, we can choose either "interactive shell mode" or "script mode" as follows.

Interactive mode:  
Figures will be displayed as the output of code cells.

In [ ]:
# Choose interactive mode (default is True when we use interactive shells)
snl.interactive = True

When you want to turn-off interactive mode temporally, set `False` as `Scenario.interactive` or apply `show_figure=False` as an argument of methods, including `Scenario.records()`. Methods with figures will be shown later in this tutorial.

In [ ]:
# apply "show_figures=False" to turn-off interactive mode temporally
# snl.records(show_figure=False)

Script mode:  
In script mode, figures will not be displayed. When filenames were applied to the methods as `filename` argument, figures will be saved in your local environment.

```Python
# Stop displaying figures
snl.interactive = False
# With this mode we can save figures, specifying "filename" argument
snl.records(filename="records.jpg")
```

`covsirphy` uses Matplotlib backend and `Scenario.records()` etc. accepts keyword arguments of `matplotlib.pyplot.savefig()`. For example, we can export TIFF images with high resolution by specifying `filename` (.tiff) and `dpi` argument.

```Python
snl.records(filename="records.tiff", dpi=500)
```

When we run codes as a script (eg. `python scenario_analysis.py`), only "script mode" is selectable and `Scenario.interactive` is always `False`. Figures will be saved when filenames are specified with `filename` argument.

Because some methods, including `Scenario.summary()`, return dataframes (`pandas.DataFrame`), we can save them as CSV files etc. using `.to_csv(filename, index=True)`.

We can produce filenames more easily with `Filer` class. Please refer to the scripts in [example directory of the repository](https://github.com/lisphilar/covid19-sir/tree/master/example).

```Python
filer = cs.Filer(directory="output", prefix="jpn", suffix=None, numbering="01")
filer.png("records")
# -> {"filename": "<absolute path>/output/jpn_01_records.png"}
filer.jpg("records")
# -> {"filename": "<absolute path>/output/jpn_01_records.jpg"}
filer.csv("records", index=True)
# -> {"path_or_buf": "<absolute path>/output/jpn_01_records.csv", index: True}
```

We can save files more easily with `Filer` as follows.

```Python
record_df = snl.records(**filer.png("records"))
record_df.to_csv(**filer.csv("records", index=False))
````

#### Backup/restore scenario

We have `Scenario.backup(filename)` and `Scenario.restore(filename)` to backup/restore time points and phase information. This will be helpful when we perform parameter estimation using a server and simulate with the estimated parameter values using local machines.  

Note that we need to execute `Scenario.register()` in advance to set time points with `Scenario.restore()`.

Backup information:

```Python
backupfile_dict = cs.Filer(directory="output")
snl.backup(**backupfile_dict)
```

Restore information:

```Python
backupfile_dict = cs.Filer(directory="output")
snl = cs.Scenario(country="Japan")
snl.register(jhu_data)
snl.restore(**backupfile_dict).summary()
```

After restoring information, we can skip `Scenario.trend()` and `Scenario.estimate()` (their functionalities will be explained later).

### Check records
Let's see the records at first. `Scenario.records()` method return the records as a pandas dataframe and show a line plot. Some kind of complement will be done for analysis, if necessary.

`Scenario.records()` shows the number of infected/recovered/fatal cases as default. Using `variables` argument, we can set the variables to show. Here, we check the number of confirmed/fatal/recovered cases. They are cumulative values.

In [ ]:
snl.records(variables="CFR").tail()
# This is the same as
# snl.records(variables=["Confirmed", "Fatal", "Recovered"])

The number of infected cases on date:

In [ ]:
snl.records(variables="I");
# This is the same as
# snl.records(variables=["Infected"])

All available variables can be retrieved with `variables="all"`.

In [ ]:
df = snl.records(variables="all", show_figure=False)
pprint(df.set_index("Date").columns.tolist(), compact=True)

We can specify the variables to show.

In [ ]:
snl.records(variables=["Vaccinations"]).tail()

We can calculate the number of daily new cases with `Scenario.record_diff()` method.

In [ ]:
# Acceptable variables are the same as Scenario.records()
snl.records_diff(variables="C", window=7);

`Scenario.show_complement()` method is useful to show the kinds of complement. The details of complement are explained in [Usage: datasets](https://lisphilar.github.io/covid19-sir/usage_dataset.html#The-number-of-cases-(JHU-style)) section.

In [ ]:
# Show the details of complement
snl.show_complement()

### S-R trend analysis
S-R trend analysis finds the change points of SIR-derived ODE parameters. Details will be explained in [Usage (details: phases)](https://lisphilar.github.io/covid19-sir/usage_phases.html). Phases will be separated with dotted lines. i.e. Dot lines indicate the start dates of phases.

In [ ]:
snl.trend().summary()

### Parameter estimation of ODE models
Here, we will estimate the tau value [min] (using grid search) and parameter values of SIR-derived models using [Optuna](https://github.com/optuna/optuna) package (automated hyperparameter optimization framework). As an example, we use SIR-F model. Details of models will be explained in [Usage (details: theoretical datasets)](https://lisphilar.github.io/covid19-sir/usage_theoretical.html).  

Note that ODE parameters are NOT universal and varies from model to model. Theta/kappa/rho/sigma are the model-specific parameters of SIR-F model. They are non-dimensionalized parameters of the differential equations of the models.

**We can select the model from SIR, SIRD and SIR-F model for parameter estimation. SIR-FV model (completely deprecated) and SEWIR-F model cannot be used.**

In [ ]:
# Estimate the tau value and parameter values of SIR-F model
snl.estimate(cs.SIRF)

In [ ]:
# Show the summary of parameter estimation
snl.summary()

### Evaluation of estimation accuracy
Accuracy of parameter estimation can be evaluated with RMSLE (Root Mean Squared Log Error) score.  

\begin{align*}
\mathrm{RMSLE} = \sqrt{\cfrac{1}{n}\sum_{i=1}^{n}(log_{10}(A_{i} + 1) - log_{10}(P_{i} + 1))^2}
\end{align*}

Where $A$ is the observed (actual) values, $P$ is estimated (predicted) values. Variables are $S (i=1), I (i=2), R (i=3)\ \mathrm{and}\ F (i=n=4)$ for SIR-F model. When RMSLE score is low, hyperparameter estimation is highly accurate.
Please refer to external sites, including [Medium: What’s the Difference Between RMSE and RMSLE?](https://medium.com/analytics-vidhya/root-mean-square-log-error-rmse-vs-rmlse-935c6cc1802a)

In [ ]:
# Show RMSLE scores with the number of optimization trials and runtime for phases
snl.summary(columns=["Start", "End", "RMSLE", "Trials", "Runtime"])

Additionally, we can visualize the accuracy with `Scenario.estimate_accuracy()`, specifying phase name.

In [ ]:
# Visualize the accuracy for the 2nd phase
snl.estimate_accuracy(phase="2nd")
# phase="last" means the last phases
# snl.estimate_accuracy(phase="last")

We can calculate total score for all phases using `Scenario.score()` method. Metrics can be selected from MAE, MSE, MSLE, RMSE, RMSLE and MAPE.

In [ ]:
# Get total score
# snl.score(metrics="RMSLE")
metrics_list = ["MAE", "MSE", "MSLE", "RMSE", "RMSLE", "MAPE"]
for metrics in metrics_list:
    metrics_name = metrics.rjust(len(max(metrics_list, key=len)))
    print(f"{metrics_name}: {snl.score(metrics=metrics):.3f}")

### Get parameter value
We can get the parameter values of a phase using `Scenario.get()` method.

In [ ]:
# Get parameter values
snl.get("Rt", phase="4th")

In [ ]:
# phase="last" means the last phases
snl.get("Rt", phase="last")

### Show parameter history
We can get the history of parameter values with a dataframe and a figure.

In [ ]:
# Get the parameter values as a dataframe
snl.summary(columns=[*cs.SIRF.PARAMETERS, "Rt"])

`Scenario.history()` method shows the trajectories of parameters (and the number of cases).

In [ ]:
snl.history(target="theta", show_legend=False);

In [ ]:
snl.history(target="kappa", show_legend=False);

In [ ]:
snl.history(target="rho", show_legend=False);

In [ ]:
snl.history(target="sigma", show_legend=False);

Notes on the history of $\sigma$ value in japan (last updated: 28Dec2020):  
In Japan, we experienced two waves and we are in third wave. In the first wave (Apr - May), recovery period was too long because collapse of the medical care system occurred and no medicines were found.

Sigma values: the first wave < the second wave > the third wave

However, in the second wave (Jul - Oct), recovery period appears short because we have some effective medicines (not approved, in clinical study), younger people (people un-associated to sever diseases) were infected.

In the third wave (Nov - ), older people tend to be infected and we are facing with medical collapse at this time...

### Show the history of reproduction number
$R_0$ ("R naught") means "the average number of secondary infections caused by an infected host" ([Infection Modeling — Part 1](https://towardsdatascience.com/infection-modeling-part-1-87e74645568a)). When this value is larger than 1, the infection is going around.

In [ ]:
snl.history(target="Rt", show_legend=False);

### Simulate the number of cases
We can compare the actual and simulated (with estimated parameter values) number of confirmed/infected/recovered/fatal cases using `Scenario.history()` method.

In [ ]:
# Compare the actual values and the main scenario
snl.history("Infected");

When we want to show only one scenario with all variables, we use `Scenario.simulate()` method.

In [ ]:
snl.simulate(name="Main");

### Main scenario
To investigate the effect of parameter changes, we will perform scenario analysis. In the main scenario, we will assume that the parameter values do not change after the last past phase.

i.e. If the parameter values will not be changed until 31May2022, how many cases will be? We call this scenario as "Main" scenario.

In [ ]:
# Clear future phases in Main scenario
snl.clear(name="Main")
# Add one future phase 30 days with the parameter set of the last past phase
snl.add(days=30, name="Main")
# Add one future phase until 31May2022 with the same parameter set
snl.add(end_date="31May2022", name="Main")
# Simulate the number of cases
snl.simulate(name="Main").tail()

### Medicine scenario
To investigate the effect of new medicines, we will assume that $\sigma$ will be changed in the future phases.

If $\sigma$ will be 1.2 times in 30 days, how many cases will be? We will call this scenario as "Medicine" scenario.

In [ ]:
# Calculate the current sigma value of the last phase
sigma_current = snl.get("sigma", name="Main", phase="last")
sigma_current

In [ ]:
# Sigma value will be double
sigma_new = sigma_current * 1.2
sigma_new

In [ ]:
# Initialize "Medicine" scenario (with the same past phases as that of Main scenario)
snl.clear(name="Medicine")
# Add 30 days as a new future phases with the same parameter set
snl.add(name="Medicine", days=30, sigma=sigma_current)
# Add a phase with doubled sigma value and the same end date with main date
snl.add(name="Medicine", end_date="31May2022", sigma=sigma_new)

Check summary of future phases.

In [ ]:
df = snl.summary()
df.loc[df["Type"] == "Future"]

Simulate the number of cases.

In [ ]:
snl.simulate(name="Medicine").tail();

### Prediction of parameter values
With extra datasets, we can predict the ODE parameter values in the future phases because [OxCGRT indicators](https://github.com/OxCGRT/covid-policy-tracker) (policy measures), vaccinations and so on impact on parameter values.

OxCGRT indicators are

- school_closing,
- workplace_closing,
- cancel_events, 
- gatherings_restrictions,
- transport_closing,
- stay_home_restrictions,
- internal_movement_restrictions,
- international_movement_restrictions,
- information_campaigns,
- testing_policy, and
- contact_tracing.

As method of prediction, we can select

- `method="multivariate_regression"` (default, multivariate forecasting of ODE parameter values with the other ODE parameter values and indicators), and
- `method="univariate"` (univariate forecasting of ODE parameter values).

[AutoTS](https://github.com/winedarksea/AutoTS) (library for automated time series forecasting) is used for prediction. AutoTS returns the most likely values and lower/upper values of forecasting. When we have four ODE parameters in Y, we will have $2^4=16$ scenarios in addition to "Likely" scenario.

`Scenario.estimate()` must be done in advance to get Y for training. Indicators should be registered in advance with `Scenario.register()` to get X for training.

`Scenario.predict()` predicts the parameter values of future phases.

In [ ]:
# When using Y of the main scenario for training
# From version 2.23.0-beta
snl.predict(name="Main", days=30);

### Adjust the end dates to align
To compare this scenario with the other scenarios, we should align the last end date with `Scenario.adjust_end()` because the last end date is different from the other scenarios at this time.

In [ ]:
# Adjust the last end dates
snl.adjust_end()
# Check the last phases of all scenarios
all_df = snl.summary().reset_index()
for name in all_df["Scenario"].unique():
    df = snl.summary(name=name)
    last_end_date = df.loc[df.index[-1], "End"]
    print(f"{name} scenario: to {last_end_date}")

### Compare the scenarios
We will compare the scenarios with representative values, reproduction number and parameter values. Currently, we can compare the scenarios with the following indexes.

- max(Infected): max value of Infected
- argmax(Infected): the date when Infected shows max value
- Infected on …: Infected on the end date of the last phase
- Fatal on …: Fatal on the end date of the last phase

In [ ]:
snl.describe()

We have too many scenarios. To discuss the forecasted number of cases deeply, we will focus on baseline ("Main"), "Medicine", the most likely, the best, the worst scenario. First, rename "Multivariate_regression_Likely" with "Likely".

In [ ]:
# From 2.23.1-theta
snl.rename(old="Multivariate_regression_Likely", new="Likely");

Next, we will find the representative scenarios. `Scenario.represent(q, variable, date=None, included=None, excluded=None)` (default) returns the names of scenarios which have the quantile values as `variable` on `date` (`None` means the last end date). All scenarios will be the target for quantile calculation when `included=None` and `excluded=None`. Applying lists to the arguments, users can specify the target.

Here, "Main" and "Likely" will be excluded from the target scenarios for quantile calculation so that we find the representative scenarios from the 16 scenarios which were created with the combinations of the upper/lower values.

We define here "scenario with 0.05-quantile Fatal on the last end date (= (today) + (days to predict))" as the best scenario. "Scenario with 0.95-quantile Fatal on the last end date" as the worst scenario.

In [ ]:
# From 2.23.1-iota
best, worst = snl.represent(q=[0.05, 0.95], variable="Fatal", date=None, excluded=["Main", "Likely"])
print({"best": best, "worst": worst})

Rename them with `Scenario.rename()`.

In [ ]:
# From 2.23.1-theta
snl.rename(old=best, new="Best")
snl.rename(old=worst, new="Worst");

Then, delete the un-necessary scenarios (i.e. scenarios represented by the best/worst scenarios) with regular expressions.

In [ ]:
# From 2.23.1-theta
snl.delete_matched(pattern=r"^Multi")

Finally, check the representative values of the scenarios with `Scenario.describe()`.

In [ ]:
snl.describe()

Show summary of future phases. The most likely scenario and scenarios with combinations of the predicted upper/lower ODE parameter values.

In [ ]:
df = snl.summary()
df.loc[df["Type"] == "Future"]


Simulate the number of cases of the most likely scenario.

In [ ]:
snl.simulate(variables="CFR", name="Likely").tail();

### History of a variable/parameter

Compare the number of cases of the all scenario with `Scenario.history()` and variable name.

In [ ]:
snl.history("Infected");

We can focus on the values in specified date range with the following arguments.

- `dates`: tuple of start date and end date
- `past_days` (integer): how many past days to use in calculation from today (`Scenario.today` property)
- `phases` (list of str): phase names to use in calculation

These arguments are effective with `Scenario.history()`, `Scenario.simulate()`, `Scenario.track()` and `Scenario.score()`.

In [ ]:
# Get the minimum value (from today to future) to set lower limit of y-axis
lower_limit = snl.history("Infected", dates=(snl.today, None), show_figure=False).min().min()
# From today to future (no limitation regarding end date)
snl.history("Infected", dates=(snl.today, None), ylim=(lower_limit, None));

In the past 20 days. Reference date is today (`Scenario.today` property).

In [ ]:
snl.history("Infected", past_days=20);

In the selected phases. Here, we will show the 3rd, 4th and 5th phase.

In [ ]:
snl.history("Infected", phases=["3rd", "4th", "5th"]);

In [ ]:
snl.history(target="Infected");

We can also compare the values of reproduction number and ODE parameters.

In [ ]:
snl.history(target="Rt");

In [ ]:
snl.history(target="rho");

In [ ]:
snl.history(target="sigma");

In [ ]:
snl.history(target="theta");

In [ ]:
snl.history(target="kappa");

### Change rate of parameters in main scenario
History of each parameter will be shown. Values will be divided by the values in 0th phase.

In [ ]:
snl.history_rate(name="Main");

### Retrospective analysis
We can evaluate the impact of measures using past records. How many people were infected if the parameter values have not changed since 01Sep2020?

In [ ]:
# Perform retrospective analysis
snl_retro = cs.Scenario(country="Japan")
snl_retro.register(jhu_data)
snl_retro.retrospective(
    "01Jan2021", model=cs.SIRF, control="Main", target="Retrospective", timeout=10)

In [ ]:
# Show the summary of estimation
cols = ["Start", "End", "ODE", "Rt", *cs.SIRF.PARAMETERS] + ["RMSLE", "Trials", "Runtime"]
snl_retro.summary(columns=cols)

In [ ]:
# History of reproduction number
snl_retro.history("Rt");

In [ ]:
# History of Infected
snl_retro.history("Infected");

In [ ]:
# Show the representative values
snl_retro.describe()